In [179]:
import spacy
from spacy.matcher import DependencyMatcher
from spacy.displacy import parse_deps
import spacy.displacy as displacy
import sys

from shared import tqdm_readlines

In [27]:
deu_nlp = spacy.load("de_zdl_lg")
rus_nlp = spacy.load("ru_core_news_md")

In [135]:
deu_snts = [deu_nlp(ln.strip().split("\t")[1]) for (ln, _) in zip(tqdm_readlines("../data/tatoeba/deu.tsv"), range(100000))]
rus_snts = [rus_nlp(ln.strip().split("\t")[1]) for (ln, _) in zip(tqdm_readlines("../data/tatoeba/rus.tsv"), range(100000))]

 17%|█████████▊                                               | 100000/579156 [05:08<24:38, 323.98it/s]


In [223]:
s = rus_snts[182]
print(s)
print(parse_deps(s))

# print(s[-3].pos_)

s = rus_nlp("Он сказал, что живёт в городе.")
print(parse_deps(s))
print(s[1].morph)

s = deu_nlp("Du musst dich zusammenreißen.")
print(parse_deps(s))
# print(s[1].morph)

Я не могу больше терпеть этот холод.
{'words': [{'text': 'Я', 'tag': 'PRON', 'lemma': None}, {'text': 'не', 'tag': 'PART', 'lemma': None}, {'text': 'могу', 'tag': 'VERB', 'lemma': None}, {'text': 'больше', 'tag': 'ADV', 'lemma': None}, {'text': 'терпеть', 'tag': 'VERB', 'lemma': None}, {'text': 'этот', 'tag': 'DET', 'lemma': None}, {'text': 'холод.', 'tag': 'NOUN', 'lemma': None}], 'arcs': [{'start': 0, 'end': 2, 'label': 'nsubj', 'dir': 'left'}, {'start': 1, 'end': 2, 'label': 'advmod', 'dir': 'left'}, {'start': 3, 'end': 4, 'label': 'advmod', 'dir': 'left'}, {'start': 2, 'end': 4, 'label': 'xcomp', 'dir': 'right'}, {'start': 5, 'end': 6, 'label': 'det', 'dir': 'left'}, {'start': 4, 'end': 6, 'label': 'obj', 'dir': 'right'}], 'settings': {'lang': 'ru', 'direction': 'ltr'}}
{'words': [{'text': 'Он', 'tag': 'PRON', 'lemma': None}, {'text': 'сказал,', 'tag': 'VERB', 'lemma': None}, {'text': 'что', 'tag': 'SCONJ', 'lemma': None}, {'text': 'живёт', 'tag': 'VERB', 'lemma': None}, {'text': '

In [155]:
for i in range(200):
    print(i, deu_snts[i])

0 Lass uns etwas versuchen!
1 Herzlichen Glückwunsch zum Geburtstag, Muiriel!
2 Muiriel ist jetzt 20.
3 Das Passwort ist "Muiriel".
4 Ich werde bald zurück sein.
5 Ich finde keine Worte.
6 Das wird nie enden.
7 Ich weiß einfach nicht, was ich sagen soll.
8 Ich war in den Bergen.
9 Ich weiß nicht, ob ich Zeit dazu habe.
10 Letztendlich muss jeder selbst lernen.
11 Das wird nichts ändern.
12 Ich verdiene 100 Euro pro Tag.
13 Es kann sein, dass ich bald aufgebe und stattdessen ein Nickerchen mache.
14 Das wird nicht passieren.
15 Manchmal kann er ein seltsamer Junge sein.
16 Ich werde mir Mühe geben, dich nicht beim Lernen zu stören.
17 Ich kann mich nur fragen, ob es für alle anderen dasselbe ist.
18 Ich vermute, das ist anders, wenn du darüber längerfristig nachdenkst.
19 Ich werde sie morgen anrufen, wenn ich zurückkomme.
20 Ich mochte mysteriöse Charaktere schon immer mehr.
21 Du solltest schlafen.
22 Ich sagte ihnen, sie sollen mir ein neues Ticket schicken.
23 Du bist so ungeduldig 

In [148]:
def search_nlp(nlp, ptn):
    matcher = DependencyMatcher(nlp.vocab)
    matcher.add("PTN", [ptn])
    return matcher

# deu_matcher = DependencyMatcher(deu_nlp.vocab)
# rus_matcher = DependencyMatcher(rus_nlp.vocab)

# rus_matcher.add("PTN", [pattern])

In [139]:
mtch = search_nlp(deu_nlp, [{
    "RIGHT_ID": "anchor", 
    "RIGHT_ATTRS": { "POS": "NOUN", "MORPH": { "IS_SUPERSET": ["Case=Dat"] } }
}, {
    "LEFT_ID": "anchor", 
    "REL_OP": ">", 
    "RIGHT_ID": "prep", 
    "RIGHT_ATTRS": {
        "DEP": "case", 
        "LEMMA": { 
            "IN": ["an", "in", "am", "im", "auf"] 
        }
    }
}])

for s, i in zip(deu_snts, range(99999)):
    m = mtch(s)
    if len(m) > 0:
        # print(i, s, m, s[m[0][1][0]])
        # print(m)
        print(i, s[m[0][1][1]], "+", s[m[0][1][0]], "|", s)
        

124 in + Händen | Sie ist in guten Händen.
265 an + Fingern | Das kann man sich an den fünf Fingern abzählen.
411 in + Dorf | Können Sie mir erklären, warum es in beinahe jedem Dorf zwei Kirchen gibt?
433 auf + Bahnhöfen | Während des Obon-Festes gibt es viel Durcheinander auf den Bahnhöfen.
1255 in + Klimaten | Reis wächst in warmen Klimaten.
1303 in + Jahren | Japan hat sich in den letzten 50 Jahren stark verändert.
1346 im + Jahre | Zuerst kamen die Kelten im Jahre 600 v. Chr.
1393 in + 1860ern | Frankfurter wurden das erste Mal in den 1860ern in den Vereinigten Staaten verkauft.
1922 in + Tagen | Ich fühle mich in den letzten Tagen nicht so gut.
2357 in + Zweigen | Ein Eichhörnchen versteckte sich in den Zweigen.
2460 im + Nu | Er hat die Arbeit im Nu beendet.
2515 in + 1970ern | Dieses Lied war in den 1970ern beliebt.
2550 in + Jahren | Das erste Mal in zehn Jahren ging ich zurück in meine Heimatstadt.
2671 In + Sinne | In gewissem Sinne ist das Leben nur ein Traum.
2888 in + Jahr

In [140]:
# Query exceptional words taking -у in prepositional case
mtch = search_nlp(rus_nlp, [
{
    "RIGHT_ID": "anchor",
    "RIGHT_ATTRS": { 
        "POS": "NOUN", 
        "MORPH": { "IS_SUPERSET": ["Case=Loc"] },
        "TEXT": { "REGEX": ".+у$" }
    }
}, 
{
    "LEFT_ID": "anchor", 
    "REL_OP": ">", 
    "RIGHT_ID": "prep", 
    "RIGHT_ATTRS": {
        "DEP": "case", 
        "LEMMA": { 
            "IN": ["в"] 
        }
    }
}])

for s, i in zip(rus_snts, range(99999)):
    m = mtch(s)
    if len(m) > 0:
        # print(i, s, m, s[m[0][1][0]])
        # print(m)
        print(i, s[m[0][1][1]], "+", s[m[0][1][0]], "|", s)
        

306 во + рту | Он говорил с трубкой во рту.
468 в + лесу | Мы не видели в лесу много оленей.
486 в + году | Я приехал в Японию в прошлом году.
578 в + саду | Я посадил в своем саду яблоню.
690 в + лесу | Она медленно скрылась в туманном лесу.
778 в + аэропорту | Я сейчас в аэропорту.
1015 в + лесу | Туристам, чтобы найти дорогу в лесу, нужно брать с собой компас.
1223 в + году | Полное солнечное затмение можно будет наблюдать в следующем году 22 июня.
1305 в + лесу | Он заблудился в лесу.
1779 в + году | Урожай в прошлом году не удался.
2018 В + саду | В саду совсем не было роз.
2032 В + году | В 1951 году сестру Терезу направили в Калькутту, крупнейший город Индии в то время.
2138 в + году | Это счастливое время в году.
2341 в + саду | Розы в саду сладко пахнут.
3094 в + году | Том занимается организацией теннисного турнира в этом году.
3377 в + лесу | Почему ты заблудился в лесу?
3454 в + году | Мой дядя уехал в Мексику в 1983 году и так и не вернулся.
3690 в + году | Колумб открыл А

In [176]:
# Query exceptional words taking -у in prepositional case
mtch = search_nlp(deu_nlp, [
{
    "RIGHT_ID": "anchor",
    "RIGHT_ATTRS": { 
        "POS": "NOUN", 
        "MORPH": { "IS_SUPERSET": ["Case=Dat", "Number=Sing"] },
        "TEXT": { "REGEX": ".+en$" },
        "LEMMA": { "REGEX": { "NOT_IN": [".+en$"] } }
    }
}
])

for s, i in zip(deu_snts, range(99999)):
    m = mtch(s)
    if len(m) > 0:
        # print(i, s, m, s[m[0][1][0]])
        # print(m)
        print(i, s[m[0][1][0]], "|", s)
        

6630 Patienten | Der Arzt hat dem Patienten ein Medikament verschrieben.
14809 Herzen | Nimm's dir nicht so zu Herzen.
21944 Patienten | Der Chirurg entfernte dem Patienten den Blinddarm.
28021 Jungen | Sie küsste dem Jungen die Tränen weg.
42988 Ohren | Mir kam zu Ohren, dass er gestorben sei.
48585 Menschen | Man kann einem Menschen nichts beibringen. Er kann es sich einzig selbst beibringen.
48643 Menschen | Die Vernunft entdeckt dem Menschen den Sinn und die Bedeutung seines Lebens.
62005 Reisenden | Der Mann entwendete dem Reisenden das Geld.
62006 Reisenden | Der Mann raubte dem Reisenden das Geld.
69749 Ohren | Mir kam zu Ohren, Tom wolle heiraten.
70200 Ganzen | Ich verpasse dem Ganzen gerade den letzten Schliff.
73452 Jungen | Sie sah dem Jungen seine Unartigkeit nach.
78205 Ganzen | Das Schwimmbecken im Garten setzte dem Ganzen die Krone auf.
92803 Ohren | Mir kam ein interessantes Gerücht zu Ohren.
94243 Menschen | Es ist schmerzlich, einem Menschen seine Grenzen anzusehen.


In [181]:
s = deu_nlp("Die Tür ist kaputt und ich werde sie bald reparieren lassen.")
print(s[7].morph)
print(parse_deps(s))
displacy.render(s, style='dep',jupyter=True)

Case=Acc|Gender=Fem|Number=Sing|Person=3|PronType=Prs
{'words': [{'text': 'Die', 'tag': 'DET', 'lemma': None}, {'text': 'Tür', 'tag': 'NOUN', 'lemma': None}, {'text': 'ist', 'tag': 'AUX', 'lemma': None}, {'text': 'kaputt', 'tag': 'ADJ', 'lemma': None}, {'text': 'und', 'tag': 'CCONJ', 'lemma': None}, {'text': 'ich', 'tag': 'PRON', 'lemma': None}, {'text': 'werde', 'tag': 'AUX', 'lemma': None}, {'text': 'sie', 'tag': 'PRON', 'lemma': None}, {'text': 'bald', 'tag': 'ADV', 'lemma': None}, {'text': 'reparieren', 'tag': 'VERB', 'lemma': None}, {'text': 'lassen.', 'tag': 'VERB', 'lemma': None}], 'arcs': [{'start': 0, 'end': 1, 'label': 'det', 'dir': 'left'}, {'start': 1, 'end': 3, 'label': 'nsubj', 'dir': 'left'}, {'start': 2, 'end': 3, 'label': 'cop', 'dir': 'left'}, {'start': 4, 'end': 10, 'label': 'cc', 'dir': 'left'}, {'start': 5, 'end': 10, 'label': 'nsubj', 'dir': 'left'}, {'start': 6, 'end': 10, 'label': 'aux', 'dir': 'left'}, {'start': 7, 'end': 10, 'label': 'obj', 'dir': 'left'}, {'s

# Some notes

We need to distinguish between different kinds of sentence puzzles involving occluded words:

- give multiple choice options for the word
- give the lemmatized form of the word, unconjugated
- blank out the word
- delete the word, so it's not even clear where it was

In [220]:
RUSSIAN_PTNS = {
    
    "RUSSIAN_PREP_Y": {
        "spacy_ptn": [{
            "RIGHT_ID": "anchor",
            "RIGHT_ATTRS": { 
                "POS": "NOUN", 
                "MORPH": { "IS_SUPERSET": ["Case=Loc"] },
                "TEXT": { "REGEX": ".+у$" }
            }
        }]
    },
    
    "RUSSIAN_PREP_LOC": {
        "spacy_ptn": [{
                "RIGHT_ID": "anchor", 
                "RIGHT_ATTRS": { 
                    "POS": "NOUN", 
                    "MORPH": { "IS_SUPERSET": ["Case=Loc"] } 
                }
            }, {
                "LEFT_ID": "anchor", 
                "REL_OP": ">", 
                "RIGHT_ID": "prep", 
                "RIGHT_ATTRS": {
                    "DEP": "case", 
                    "LEMMA": { 
                        "IN": ["в", "на"] 
                    }
                }
        }]
    },

    "RUSSIAN_NO_BACKSHIFT": {
        "spacy_ptn": [{
                "RIGHT_ID": "anchor", 
                "RIGHT_ATTRS": { 
                    "POS": "VERB", 
                    "MORPH": { "IS_SUPERSET": ["Tense=Past", "Mood=Ind"] } 
                }
            }, {
                "LEFT_ID": "anchor", 
                "REL_OP": ">", 
                "RIGHT_ID": "verb2", 
                "RIGHT_ATTRS": {
                    "DEP": "ccomp", 
                    "MORPH": { "IS_SUPERSET": ["Tense=Pres"] } 
                }
        }]
    },
}

def find_snt_matches(nlp, ptns, snts):
    all_matches = []
    
    matcher = DependencyMatcher(nlp.vocab)
    for (ptn_name, ptn) in ptns.items():
        ptn_id = matcher.add(ptn_name, [ptn["spacy_ptn"]])
        print(ptn_id)

    for (i, s) in zip(range(len(snts)), snts):
        ms = matcher(s)
        for i in range(len(ms)):
            ms[i] = (nlp.vocab.strings[ms[i][0]], ms[i][1])
        if len(ms) > 0:
            res = {
                "index": i,
                "sentence": s,
                "match": ms
            }
            all_matches.append(res)

    return all_matches

res = find_snt_matches(rus_nlp, RUSSIAN_PTNS, rus_snts)

None
None
None


In [222]:
for i in range(400):
    r = res[i]
    # print(r)
    if "RUSSIAN_NO_BACKSHIFT" in [x[0] for x in r["match"]]:
        print(r)

{'index': 0, 'sentence': Я спросил у него, как его зовут., 'match': [('RUSSIAN_NO_BACKSHIFT', [1, 7])]}
{'index': 0, 'sentence': Я спросил, где она живёт., 'match': [('RUSSIAN_NO_BACKSHIFT', [1, 5])]}
{'index': 0, 'sentence': Он услышал, как его зовут по имени., 'match': [('RUSSIAN_NO_BACKSHIFT', [1, 5])]}
{'index': 0, 'sentence': Я увидел, что ко мне идёт человек., 'match': [('RUSSIAN_NO_BACKSHIFT', [1, 6])]}
{'index': 0, 'sentence': Джон сказал, что хочет жениться на мне, и я счастлива как никогда., 'match': [('RUSSIAN_NO_BACKSHIFT', [1, 4])]}
{'index': 0, 'sentence': Мне всегда казалось, что ты поедешь с нами., 'match': [('RUSSIAN_NO_BACKSHIFT', [2, 6])]}
{'index': 0, 'sentence': Вы когда-нибудь слушали, как она играет на пианино?, 'match': [('RUSSIAN_NO_BACKSHIFT', [4, 8])]}
{'index': 0, 'sentence': Мы верили, что Земля вращается вокруг Солнца., 'match': [('RUSSIAN_NO_BACKSHIFT', [1, 5])]}
{'index': 0, 'sentence': Он взглянул на неё и увидел, что она сердится., 'match': [('RUSSIAN_